In [31]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = 30  # Which month and days result you want to visualize in table
month = 10
repetitionValue = 23
repetitionName = "x"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

51
5.666666666666667


In [32]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]

                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])

                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 387)
(10, 387)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_17312\3429005192.py:170: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage / nrOfSubjects


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,0.516667,None,0.633333,None,None,None,None,None,None,0.516667,0.583333,0.516667,0.516667,None,0.566667,None,None,0.566667,0.51875,None,0.58125,None,None,None,0.560417,None,0.629167,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.564583,None,None,0.539583,None,None,0.510417,None,None,0.5375,None,None,None,None,None,None,None,None,None,None,None,0.541667,None,None,0.539583,0.627083,0.51875,None,0.55,None,0.660417,0.529167,None,0.672917,0.572917,0.583333,0.579167,0.552083,0.55,0.679167,0.5,None,0.595833,None,None,None,0.54375,None,None,0.5375,None,0.5875,0.545833,None,0.589

In [33]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.7 : subject 5, feature:dataFFTCV-dataHR--fftData, kernellinear, C:0.5
Accuracy 0.68 : subject 1, feature:dataHRCV-dataWCV--dataHR, kernelrbf, C:2.5
Accuracy 0.68 : subject 1, feature:dataCV2-dataCV--dataHR, kernelrbf, C:2.5
Accuracy 0.68 : subject 7, feature:dataHRCV-dataWCV--welchData, kernellinear, C:0.5
Accuracy 0.68 : subject 1, feature:dataCV2-dataWCV--dataHR, kernelrbf, C:2.5
Accuracy 0.68 : subject 1, feature:dataCV2--dataHR, kernelrbf, C:2.5
Accuracy 0.68 : subject 9, feature:dataWCV--welchData, kernelsigmoid, C:2.5
Accuracy 0.68 : subject 4, feature:dataCV2-dataWCV--welchData, kernellinear, C:0.5
Accuracy 0.67 : subject 1, feature:dataHRCV--dataHR, kernelrbf, C:2.5
Accuracy 0.67 : subject 5, feature:dataCV2-dataFFTCV--welchData, kernellinear, C:0.5


In [34]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.55738225 0.53489023 0.54264601 0.54660022 0.5688933
 0.53789203 0.55475788 0.54773504 0.54258041]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386
0,0.523864,0.545139,0.54048,0.593955,0.561111,0.522475,0.533519,0.533426,0.558056,0.511574,0.553241,0.565062,0.530556,0.522447,0.521188,0.547222,0.541919,0.546667,0.539949,0.536225,0.568056,0.552715,0.56501,0.532323,0.542139,0.521285,0.534486,0.551513,0.559175,0.558787,0.532569,0.544691,0.558981,0.572798,0.552222,0.53559,0.568616,0.537616,0.568674,0.535922,0.526042,0.550691,0.546971,0.544429,0.52207,0.570431,0.5625,0.533901,0.557292,0.526042,0.541493,0.589769,0.51397,0.525351,0.560995,0.576389,0.540421,0.572963,0.569366,0.535448,0.594216,0.569669,0.540833,0.57816,0.563364,0.544358,0.558803,0.535861,0.534262,0.529308,0.537263,0.524769,0.580961,0.556667,0.590856,0.573148,0.517483,0.571701,0.528977,0.501736,0.57776,0.546898,0.547205,0.536583,0.550833,0.53375,0.57,0.554282,0.562886,0.573997,0.539352,0.544444,0.57581,0.525347,0.54596,0.557292,0.539468,0.554956,0.558866,0.555177,0.590104,0.558556,0.545718,0.531441,0.522666,0.53816,0.575949,0.541275,0.562077,0.593056,0.530729,0.524531,0.538773,0.526042,0.542139,0.560648,0.542477,0.567077,0.517081,0.522135,0.582334,0.568608,0.535778,0.547199,0.533889,0.552431,0.563731,0.537338,0.568796,0.537778,0.530106,0.555669,0.552309,0.518734,0.564167,0.52819,0.547708,0.544876,0.527778,0.532513,0.551513,0.545128,0.558178,0.536209,0.546253,0.577616,0.578495,0.545472,0.539804,0.572213,0.536458,0.527351,0.536817,0.566165,0.51553,0.615625,0.561111,0.562384,0.532569,0.540793,0.541632,0.528657,0.539722,0.567031,0.525347,0.54239,0.539583,0.556944,0.546083,0.540104,0.57909,0.556327,0.536667,0.572798,0.557326,0.538773,0.565499,0.581206,0.557818,0.548681,0.536458,0.540785,0.554003,0.522832,0.561082,0.556016,0.547585,0.560425,0.535922,0.525682,0.532431,0.546971,0.539468,0.534312,0.570431,0.538786,0.543355,0.557292,0.526042,0.538237,0.541667,0.590278,0.518824,0.553859,0.508996,0.551143,0.527902,0.547205,0.524792,0.539131,0.53171,0.517619,0.530408,0.535822,0.505,0.514244,0.523611,0.512942,0.623071,0.547292,0.576813,0.589769,0.528848,0.532697,0.569707,0.566574,0.541597,0.563669,0.527795,0.54275,0.550116,0.534468,0.541856,0.588183,0.536539,0.532517,0.554977,0.552951,0.568411,0.554055,0.565522,0.533352,0.568392,0.552431,0.552662,0.591922,0.589962,0.575955,0.543596,0.57691,0.56669,0.601293,0.567835,0.583056,0.586481,0.550781,0.52244,0.589555,0.574705,0.556014,0.529308,0.546296,0.528472,0.57375,0.561136,0.542477,0.592751,0.538644,0.559607,0.537633,0.527072,0.537614,0.5825,0.565387,0.531081,0.55,0.546441,0.541458,0.525046,0.55119,0.525185,0.580961,0.616435,0.559571,0.596672,0.563333,0.553429,0.585475,0.587569

Max average accuracy 0.6520833333333332
314
1.0
dataCV2-dataHRCV--dataHR
sigmoid
C: 2.5


In [35]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386
0,2.619318,1.090278,1.621439,4.157685,4.488889,2.612374,2.667593,1.600278,1.674167,2.046296,2.212963,1.695185,2.652778,3.13468,3.12713,1.641667,2.709596,2.733333,2.699747,2.144899,1.136111,2.210859,2.825051,1.59697,2.710694,1.042569,2.672431,3.30908,2.795875,2.793937,3.727986,1.634074,1.676944,2.291193,2.208889,1.071181,2.274463,1.612847,2.843371,2.679609,1.578125,2.753453,3.281829,2.177715,3.132418,2.281723,1.6875,3.737309,1.114583,1.578125,1.082986,1.769306,2.05588,3.152106,2.804977,1.152778,3.242527,4.010741,3.416193,2.677241,4.159514,3.987683,2.704167,3.468958,2.253457,1.633074,4.470427,2.679306,3.205574,3.705154,2.149051,1.574306,2.323843,1.67,1.181713,2.292593,2.069931,2.286806,2.644884,2.006944,2.311042,1.640694,2.188819,2.682917,1.6525,1.60125,1.71,3.325694,1.688657,1.721991,1.078704,1.633333,1.727431,1.050694,2.183838,1.671875,2.15787,2.219823,2.794329,2.220707,1.180208,2.792778,2.728588,2.125764,2.613331,2.152639,2.879745,3.247652,2.810385,1.186111,2.122917,3.671717,1.616319,2.104167,2.710694,1.681944,2.712384,1.134154,3.102487,2.088542,1.747001,2.274432,3.214668,1.641597,1.601667,1.657292,2.818655,1.612014,1.706389,2.688889,2.120423,3.334015,2.209236,2.074937,2.820833,2.640949,1.095417,2.724379,1.055556,1.597538,3.30908,2.725642,2.79089,4.289672,3.823771,2.888081,3.470972,3.272832,2.159217,4.005492,2.682292,3.164104,2.684085,1.698495,1.031061,1.846875,1.683333,1.687153,3.727986,2.163171,2.166528,3.171944,1.619167,2.268125,1.576042,2.16956,1.61875,1.113889,2.730417,1.080208,1.737269,1.668981,1.61,2.291193,1.114653,1.077546,2.261995,2.906029,1.673453,2.743403,0.536458,2.163142,2.770013,1.568497,2.244329,2.224064,1.642755,2.241698,2.679609,1.051364,3.194583,3.281829,2.697338,2.671559,2.281723,2.155145,3.260133,1.114583,0.526042,1.61471,2.708333,1.180556,2.075297,2.215436,2.035985,2.204571,3.167412,2.188819,1.049583,2.695654,2.126841,3.105713,1.591225,3.214931,1.01,2.056976,2.618056,1.538826,1.869213,1.641875,1.73044,1.769306,2.115394,2.130787,1.70912,1.699722,2.166389,2.254676,2.111181,2.71375,1.650347,1.603403,2.709282,2.352731,2.146157,2.130069,2.774884,1.105903,2.842056,3.878388,3.393131,2.666761,2.273569,2.209722,1.657986,2.367687,1.769886,2.303819,1.630787,1.153819,1.700069,4.209051,3.407008,1.749167,3.518889,2.203125,2.08976,4.126888,1.724116,2.780069,3.705154,1.092593,1.056944,4.01625,2.244545,2.712386,3.556509,3.231862,2.238428,2.150532,2.108287,1.612841,2.9125,2.261547,2.655404,0.55,2.185764,1.082917,2.625231,2.755949,1.575556,2.323843,1.849306,1.678714,2.38669,2.253333,2.767146,2.341898,1.175139,1.1354

4
8.0
Max total significant accuracy then averaged 0.5611111111111112 
-welchData
sigmoid
C: 2.5 


In [36]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
